In [ ]:
%pip install wandb -Uqq

In [ ]:
%pip install nmslib -Uqq

     |████████████████████████████████| 13.5 MB 5.1 MB/s 
     |████████████████████████████████| 188 kB 47.6 MB/s 


In [ ]:
%pip install LibRecommender -Uqq

     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 24.1 MB 1.6 MB/s 


In [ ]:
%pip install faiss==1.5.3 -Uqq

     |████████████████████████████████| 4.7 MB 4.9 MB/s 


In [ ]:
%pip install ujson redis -Uqq

     |████████████████████████████████| 45 kB 2.2 MB/s 
     |████████████████████████████████| 246 kB 8.7 MB/s 


In [ ]:
import wandb
wandb.login()

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from libreco.data import random_split, split_by_ratio_chrono, DatasetPure
from libreco.algorithms import Caser   # pure data
from libreco.evaluation import evaluate

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from libserving.serialization import save_embed

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/00-MTC/models/Caser" 
if not os.path.isdir(MODEL_PATH):
    os.mkdir(MODEL_PATH)  
MODEL_NAME = 'Caser'

In [ ]:
data = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
data.columns = ["user", "item", "label", "time"]
data["label"] = 1 

train_data, eval_data, test_data = split_by_ratio_chrono(data, multi_ratios=[0.8, 0.1, 0.1])

train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
eval_data.build_negative_samples(data_info) 
test_data.build_negative_samples(data_info)
print(data_info)   # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %

random neg item sampling elapsed: 0.100s
random neg item sampling elapsed: 0.011s
random neg item sampling elapsed: 0.010s
n_users: 610, n_items: 8237, data sparsity: 1.6056 %


In [ ]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/00-MTC/models/Caser" 
MODEL_NAME = 'Caser'

sweep_config = {
    "name": "Caser-sweep",
    "metric": {"name": "loss", "goal": "minimize"},    
    "method": "random",  
    'early_terminate':{
      'type': 'hyperband',
      'min_iter': 10
  },  
    "parameters": {        
          "embed_size": {
              "values": [16, 32]
          },
          'n_epochs': {
              "values": [5,10]
          }, 
          "lr": {
              "values": [0.1, 0.01, 0.001, 0.0001]
          },
                    
          "batch_size": {
              "values": [64, 128, 256]
          },
          }
}
metrics = ["loss", "roc_auc", "precision", "recall", "map", "ndcg"]    
sweep_id = wandb.sweep(sweep_config, 
                       project="recsys-app", 
                       entity="iloncka-ds")

os.environ["WANDB_RUN_GROUP"] = "experiment-Caser"

def train_with_wandb():  
      with wandb.init(group=MODEL_NAME,
                      notes=f"{MODEL_NAME} hyperparameters search",
                      tags=["embed", "sequence", "hyperparameters"]) as runwb:
        # Overwrite the random run names chosen by wandb
        sc = runwb.config
        name_str = f'{MODEL_NAME}_met_{sweep_config["method"]}' + \
          f'_n_epoch_{sc.n_epochs}_' + \
          f'emb_size_{sc.embed_size}_lr_{sc.lr}_' + \
          f'batch_size_{sc.batch_size}'
        runwb.config.update({"name": name_str})
        current_dir = os.path.join(MODEL_PATH, name_str)
        if not os.path.isdir(current_dir):
            os.mkdir(current_dir)  
        # Rest of code.
        model = Caser(task="ranking", data_info=data_info, 
                        embed_size=sc.embed_size, n_epochs=sc.n_epochs, 
                        lr=sc.lr, reg=None, batch_size=sc.batch_size, lr_decay=True,)
        #NB!! param runwb is absent in official version of LibRecommender, for training unpublished changes to this library were used
        model.fit(runwb, train_data, verbose=2, eval_data=eval_data, metrics=metrics)
        #NB!! param runwb is absent in official version of LibRecommender, for training unpublished changes to this library were used
        evaluate(runwb=runwb, model=model, data=test_data,
                                    metrics=metrics)
        # specify model saving directory
        save_embed(current_dir, model)        
        model.save(path=current_dir, model_name="Caser", manual=True, inference_only=True)
wandb.agent(sweep_id, function=train_with_wandb, entity="iloncka-ds", project="recsys-app", count=20)

Create sweep with ID: 1xnmnsi1
Sweep URL: https://wandb.ai/iloncka-ds/recsys-app/sweeps/1xnmnsi1


wandb: Agent Starting Run: umndml8d with config:
wandb: 	batch_size: 64
wandb: 	embed_size: 32
wandb: 	lr: 0.01
wandb: 	n_epochs: 10
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


total params: 826,893 | embedding params: 818,597 | network params: 8,296
Training start time: 2022-11-06 15:26:24
With lr_decay, epoch 1 learning rate: 0.009999999776482582


train: 100%|██████████| 2521/2521 [00:51<00:00, 49.16it/s]


Epoch 1 elapsed: 51.336s
	 train_loss: 0.5042


...

eval_rec: 100%|██████████| 608/608 [00:00<00:00, 1768.97it/s]


epoch,▁▂▃▃▄▅▆▆▇█
log_loss,▁▁▂▃▄▄▅▆▇█
loss,▁
map,▁
map@10,▁█▅▅█▃▁▃▃▄
ndcg,▁
ndcg@10,▂█▄▅▇▃▁▃▃▄
precision,▁
precision@10,▂█▄▄▅▂▁▃▁▃
recall,▁
recall@10,▇▇▁▇▅▆▁▃▃█


wandb: Agent Starting Run: egkyhfpo with config:
wandb: 	batch_size: 64
wandb: 	embed_size: 32
wandb: 	lr: 0.001
wandb: 	n_epochs: 5


total params: 826,893 | embedding params: 818,597 | network params: 8,296
Training start time: 2022-11-06 16:48:40
With lr_decay, epoch 1 learning rate: 0.0010000000474974513


train:  39%|███▊      | 976/2521 [00:24<00:39, 38.85it/s]

https://wandb.ai/iloncka-ds/recsys-app/groups/WaveNet/workspace?workspace=user-iloncka-ds